# Generate whole file

In [4]:
import re
import pandas as pd

import re
import pandas as pd


def extract_feats_from_markdown(markdown_file, excel_file):
    """
    Extract feats from a markdown file and save them into an Excel file.

    Expected markdown format for each feat:

    ### **Feat Name**
    *"Flavor text here."*
    - First bullet point.
    - Second bullet point.
    - etc.

    Parameters:
      markdown_file (str): Path to the markdown file.
      excel_file (str): Path to save the resulting Excel file.
    """
    # Read the markdown file; handle both Unix (\n) and Windows (\r\n) newlines.
    with open(markdown_file, "r", encoding="utf-8") as f:
        content = f.read()

    # Regex pattern:
    # Matches:
    #   1. A header line: "### **Feat Name**"
    #   2. A flavor text line starting with *" and ending with "* (on its own line)
    #   3. One or more bullet lines starting with "- "
    pattern = re.compile(
        r"### \*\*(?P<name>.*?)\*\*\r?\n\*\"(?P<flavor>.*?)\"\*\r?\n(?P<bullets>(?:- .+\r?\n)+)",
        re.MULTILINE,
    )

    feats = []
    for match in pattern.finditer(content):
        name = match.group("name").strip()
        flavor = match.group("flavor").strip()
        bullets_raw = match.group("bullets").strip()
        # Split by lines and clean the bullet markers.
        bullets = [
            line.lstrip("- ").strip()
            for line in bullets_raw.splitlines()
            if line.startswith("- ")
        ]
        feats.append(
            {"Name": name, "Flavor Text": flavor, "Features": "; ".join(bullets)}
        )

    df = pd.DataFrame(feats)
    df.to_excel(excel_file, index=False)
    print(f"Extracted {len(feats)} feats and saved to {excel_file}")


extract_feats_from_markdown("markdown/Vestigium Feats.md", "out/vestigium_feats.xlsx")

Extracted 78 feats and saved to out/vestigium_feats.xlsx


In [10]:
def extract_conditions_from_markdown(markdown_file, excel_file):
    """
    Extract conditions from a markdown file and save them into an Excel file.

    Expected markdown format:

    ### Condition Name
    Condition description text (may span multiple paragraphs)
    (Optionally, bullet points may follow)

    Each condition is separated by a header line starting with "### ".

    Parameters:
      markdown_file (str): Path to the markdown file.
      excel_file (str): Path to save the resulting Excel file.
    """
    # Read the entire markdown file.
    with open(markdown_file, "r", encoding="utf-8") as f:
        content = f.read()

    # Split the content at each header starting with "### " (using the MULTILINE flag).
    parts = re.split("---", content, flags=re.MULTILINE)
    parts = [part for part in parts if part.strip() != ""]
    # The first part before the first "###" header is usually the introduction; skip it.
    conditions = []
    for part in parts[1:]:
        # Each part should begin with the condition name on its first line.
        # The remaining text is the condition description.
        lines = part.splitlines()
        lines = [line for line in lines if line.strip() != ""]
        if not lines:
            continue
        name = re.sub(r"#", "", lines[0].strip())
        text = "\n".join(lines[1:]).strip()
        conditions.append({"Condition Name": name, "Condition Text": text})

    # Create a DataFrame and save it to Excel.
    df = pd.DataFrame(conditions)
    df.to_excel(excel_file, index=False)
    print(f"Extracted {len(conditions)} conditions and saved to {excel_file}")


extract_conditions_from_markdown(
    "markdown/Expanded Conditions.md", "out/expanded_conditions.xlsx"
)

Extracted 10 conditions and saved to out/expanded_conditions.xlsx
